In [1]:
import keras
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


### 分割成train跟test

In [2]:
data = np.load('train.npy', allow_pickle = True)[:,0:2]
print(data.shape)
y_train = np.load('train.npy', allow_pickle=True)[:,3]
label = y_train
print(y_train.shape)
data, data_test, y_train, y_test = train_test_split(data, y_train, test_size=0.2, random_state=42)
month = data[:,1]
month_test = data_test[:,1]
data = data[:,0]
data_test = data_test[:,0]
print(month.shape)

(125058, 2)
(125058,)
(100046,)


### 製作word2vec、RNN的Embedding layer

這邊也會跑很久來產生model，所以我就直接讀取之前存好的

In [3]:
#w2v_model = Word2Vec(data,min_count = 10, size=250, sg = 1, iter = 10)#, negative = 3)
#w2v_model.save("w2v.model")
w2v_model = Word2Vec.load("w2v.model")
embedding_matrix = np.zeros((len(w2v_model.wv.vocab.items()) + 1, w2v_model.vector_size))
word2idx = {}

vocab_list = [(word, w2v_model.wv[word]) for word, _ in w2v_model.wv.vocab.items()]
for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

    embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            mask_zero=True,
                            weights=[embedding_matrix],
                            trainable=True)

def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
        new_corpus.append(new_doc)
    return np.array(new_corpus)

PADDING_LENGTH = 100
x_train = text_to_index(data)
x_train = pad_sequences(x_train, maxlen=PADDING_LENGTH)
x_test = text_to_index(data_test)
x_test = pad_sequences(x_test, maxlen=PADDING_LENGTH)
print("Shape:", x_train.shape)
print("Sample:", x_train[100])

Shape: (100046, 100)
Sample: [ 151 5110 5779  180    0 5780 2605 1698    0 1371 2112 2697 5778 5781
    0 1199  100 5778    0  171 5782 5783 5784  133 1203 1074 5783 5785
 5786 5787    0 3363 5788    0 1369  382 4906 3678 1241    0  919  920
 2098 5789  151  382 4121 5790 5791  242 5792 1333 1570  977    0  900
 5734 5778 2104  100 5793  177  178  179  180  181  182  183  184  185
  186  187  188  189  190  191  192  193  194  195  196  197  198  199
  200  201  202  203  204  205  206  207  208  209  210  211  212  178
  213  214]


### 將分割好的train跟test存成npy，方便之後使用

In [ ]:
np.save('x_train.npy',x_train)
np.save('y_train.npy',y_train)
np.save('x_test.npy',x_test)
np.save('y_test.npy',y_test)

## RNN model的製作
每次的train跟test都不一樣，所以我拿之前存的

In [4]:
x_train = np.load('x_train.npy', allow_pickle = True)
x_test = np.load('x_test.npy', allow_pickle = True)
y_train = np.load('y_train.npy', allow_pickle = True)
y_test = np.load('y_test.npy', allow_pickle = True)
def rnn_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(GRU(16,activation='tanh', return_sequences = True))
    model.add(Dropout(0.3))
    model.add(GRU(16))
    model.add(Dropout(0.3))
    model.add(Dense(32))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [5]:
model = rnn_model()

### 開始train，然後利用callback function幫忙找出最優model
要train很久，一樣不跑

In [25]:
np.random.seed(1230)
model.summary()
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./model/mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')

model.fit(x=x_train, y=y_train, batch_size=10000, epochs=20, shuffle = True, class_weight={0:1.06, 1:1},
          validation_split = 0.1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_71470 (Embedding)  (None, None, 250)         17867750  
_________________________________________________________________
gru_11 (GRU)                 (None, None, 16)          12816     
_________________________________________________________________
dropout_26 (Dropout)         (None, None, 16)          0         
_________________________________________________________________
gru_12 (GRU)                 (None, 16)                1584      
_________________________________________________________________
dropout_27 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_28 (Dropout)         (None, 32)               

/home/cior5757e/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/home/cior5757e/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 90041 samples, validate on 10005 samples
Epoch 1/20
90041/90041 [==============================] - 56s 620us/step - loss: 0.7178 - accuracy: 0.5083 - val_loss: 0.7114 - val_accuracy: 0.5337
Epoch 2/20
90041/90041 [==============================] - 54s 598us/step - loss: 0.7137 - accuracy: 0.5179 - val_loss: 0.7105 - val_accuracy: 0.5368
Epoch 3/20
90041/90041 [==============================] - 55s 607us/step - loss: 0.7131 - accuracy: 0.5211 - val_loss: 0.7090 - val_accuracy: 0.5457
Epoch 4/20
90041/90041 [==============================] - 53s 593us/step - loss: 0.7110 - accuracy: 0.5338 - val_loss: 0.7064 - val_accuracy: 0.5530
Epoch 5/20
90041/90041 [==============================] - 54s 601us/step - loss: 0.7076 - accuracy: 0.5491 - val_loss: 0.7037 - val_accuracy: 0.5601
Epoch 6/20
90041/90041 [==============================] - 55s 616us/step - loss: 0.7034 - accuracy: 0.5554 - val_loss: 0.6998 - val_accuracy: 0.5684
Epoch 7/20
90041/90041 [==============================] 

### 預測test data、並印出準確率、confusion matrix
拿之前跑好的model

In [6]:
model = load_model('./model/GRU.h5')
output = model.predict(x_test)
tmp = []
for i in output:
    if i[0] > 0.5:
        tmp.append(1)
    else:
        tmp.append(0)
output = np.array(tmp)
y_test = y_test.astype('int')
y_train = y_train.astype('int')
tn, fp, fn, tp = cm(y_test, output).ravel()
result = pd.DataFrame([[tp, fp], [fn, tn]], columns = ['真實為漲', '真實為跌'])
result.index = ['預測為漲', '預測為跌']
print('整體準確率:',(sum(output == y_test)/y_test.shape)[0])
print('對於漲的資料的預測準確率:', tp/(tp+fn))
print('對於跌的資料的預測準確率:', tn/(tn+fp))
result

/home/cior5757e/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


整體準確率: 0.6027906604829681
對於漲的資料的預測準確率: 0.5950132225160559
對於跌的資料的預測準確率: 0.6115309501570858


,真實為漲,真實為跌
預測為漲,7875,4575
預測為跌,5360,7202


### 將model存起來方便之後使用

In [30]:
model.save('GRU.h5')